##Задание


1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)

2. Скачиваем готовый новостной датасет !wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

     ```
     # пример работы с ним 
     from corus import load_ods_gazeta
     path = 'gazeta.csv.gz'
     records = load_ods_gazeta(path)
     next(records)
     ```

реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу) визуально оценить качество


---

In [1]:
!pip install transformers
!pip install sacremoses
!pip install torch torchvision
!pip install corus
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 10.0 MB/s 
     |████████████████████████████████| 6.6 MB 77.0 MB/s 
     |████████████████████████████████| 101 kB 15.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 7.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=797eaedcd12d28e8db583c4bfc80158cafb9ced514a84798df49955f86689067
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.1 MB/s 
Looking in indexes: 

In [2]:
import re
import io

import transformers
import sacremoses

import corus
from corus import load_ods_gazeta


!pip install annoy
import annoy

from sentence_transformers import SentenceTransformer, util

import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 8.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=388777 sha256=47fec05867e0379aabf7daa315eb4863581dce9de6447e4b01704ba51d8b7766
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy


In [3]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-08-21 10:21:57--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  19.4MB/s    in 0.7s    

2022-08-21 10:21:58 (19.4 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]



In [4]:
path_to_file = "/content/rus-eng/rus.txt"

In [5]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [6]:
!ls /content/rus-eng/ -lah

total 71M
drwxr-xr-x 2 root root 4.0K Aug 21 10:22 .
drwxr-xr-x 1 root root 4.0K Aug 21 10:22 ..
-rw-r--r-- 1 root root 1.5K May  2 01:29 _about.txt
-rw-r--r-- 1 root root  71M May  2 01:29 rus.txt


In [7]:
f = open(path_to_file)
for line in f:
    print(line)
    break

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)



In [8]:
def preprocess_sentence(w):
    w = w.lower().strip()

    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
  
    w = w.strip()

    return w

In [9]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [10]:
en, ru = create_dataset(path_to_file, None)
print(en[1])
print(ru[1])

go .
иди .


In [11]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-ru-en"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [12]:
for i in range(10):
    input = ru[i]
    input_ids = tokenizer.encode(input, return_tensors="pt")
    output = model.generate(input_ids)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}, real: {en[i]}')

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


ru: марш !, en: march!, real: go .
ru: иди ., en: Go., real: go .
ru: идите ., en: Go., real: go .
ru: здравствуйте ., en: Hello., real: hi .
ru: привет !, en: Hello!, real: hi .
ru: хай ., en: Hai., real: hi .
ru: здрасте ., en: , real: hi .
ru: здоро во !, en: Good morning and welcome!, real: hi .
ru: приветик !, en: Hello!, real: hi .
ru: беги !, en: Run!, real: run !


In [13]:
inputs = ['хорошая погода', 'я иду в школу', 'Я иду на урок в школу', 
          'Я сам решаю трудную задачу в школе', 'Я не люблю, когда идет снег.', 
          'Сегодня я пойду гулять, а потом пойду в магазин', 'Мама приготовила вкусный обед для нашей большой семьи',
          'Мне здесь нравится, но я очень скучаю по дому. Там остались мама и папа.']
for input in inputs:
    input_ids = tokenizer.encode(input, return_tensors="pt")
    output = model.generate(input_ids)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}')

ru: хорошая погода, en: good weather
ru: я иду в школу, en: I'm going to school
ru: Я иду на урок в школу, en: I go to school for a lesson
ru: Я сам решаю трудную задачу в школе, en: I solve a difficult task at school myself
ru: Я не люблю, когда идет снег., en: I don't like it when it's snowing.
ru: Сегодня я пойду гулять, а потом пойду в магазин, en: Today I'll go for a walk and then I'll go to the store
ru: Мама приготовила вкусный обед для нашей большой семьи, en: Mum cooked a delicious lunch for our big family
ru: Мне здесь нравится, но я очень скучаю по дому. Там остались мама и папа., en: I love it here, but I miss the house very much. My mum and dad stayed there.
